In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
! ls ../../models/

cached					  ERICA--exported.tar.gz
distilroberta-finetuned			  ERICA--exported.tar.gz.tar.gz
distilroberta-finetuned__exported	  hopperbot-medium
distilroberta-finetuned__exported.tar.gz  runs
ERICA


In [3]:
MODEL_NAME = "../../models/ERICA"

In [4]:
from pathlib import Path

Path(MODEL_NAME).exists()

True

In [17]:
import argparse
import re

import torch
from transformers import (
    AutoConfig,
    AutoModelForCausalLM,
    AutoTokenizer,
    Conversation,
    ConversationalPipeline,
)

from src.chat import *
from src.classifier import Classifier
from src.utils import DATA_DIR, ROOT_DIR

In [13]:
print(DATA_DIR)
print(ROOT_DIR)

/home/jovyan/work/research/data
/home/jovyan/work/research


In [14]:
CLASSIFIER = ROOT_DIR.parent.resolve().joinpath("models").joinpath("distilroberta-finetuned")

In [15]:
Path(CLASSIFIER).exists()

True

In [18]:
import json

config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    from_tf=False,
    config=config,
)
classifier = Classifier(model=str(CLASSIFIER))

In [20]:
chat(model, tokenizer, classifier=classifier, device="cpu")

>>  I'm feeling depressed


User: I'm feeling depressed
Bot:  Why?


>>  My dad passed away


User: My dad passed away
Bot:  I'm so sorry to hear that. I lost my father recently as well. May he rest in peace.


>>  Thank you. I am sorry for your loss as well.


...

>>  I am proud of building this chatbot.


User: I am proud of building this chatbot.
Bot:  That is great, what did you learn from it so far?


>>  I learned about training bots in an effective way.


User: I learned about training bots in an effective way.
Bot:  What was the method?


KeyboardInterrupt: Interrupted by user

In [23]:
chat_pipeline(model, tokenizer, classifier=classifier, device="cpu")

>>  I'm feeling depressed


User: I'm feeling depressed
Bot:  I'm sorry to hear that. What's wrong?


>>  My dad passed away yesterday


User: My dad passed away yesterday
Bot:  I'm so sorry to hear that. I hope you are doing okay.


>>  Thanks. I am trying my best.


...

>>  I am trying to be positive.


User: I am trying to be positive.
Bot:  That's a good attitude to have.


>>  It really is. I have a more optimistic outlook on life.


User: It really is. I have a more optimistic outlook on life.
Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/opt/conda/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3398, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1084405/3896992558.py", line 1, in <cell line: 1>
    chat_pipeline(model, tokenizer, classifier=classifier, device="cpu")
  File "/home/jovyan/work/research/src/chat.py", line 82, in chat_pipeline
    result = pipe(conversation)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/conversational.py", line 239, in __call__
    outputs = super().__call__(conversations, num_workers=num_workers, **kwargs)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py", line 1026, in __call__
    return self.run_single(inputs, preprocess_params, forward_params, postprocess_params)
  File "/opt/conda/lib/python3.10/site-packages/transformers/pipelines/base.py", line 1033, in run_single
    model_outputs = self.forward(model_inputs, **forward_p

In [ ]:
#check the size of the hist, if it's over 'max_lenght' remove the oldest tokens
            if bot_input_ids.size(dim=1) >= args.get('max_length'):
                #trim the tokens
                bot_input_ids = torch.narrow(bot_input_ids, 1, -args.get('max_length'), args.get('max_length'))
            chat_history_ids = model.generate(
                bot_input_ids,
                max_length=args.get('max_length')+20,
                pad_token_id=tokenizer.eos_token_id,
                no_repeat_ngram_size=args.get('no_repeat_ngram_size'),
                do_sample=args.get('do_sample'),
                top_k=args.get('top_k'),
                top_p=args.get('top_p'),
                temperature=args.get('temperature')
                )

# Issues with DialoGPT (so many)

- https://github.com/microsoft/DialoGPT/issues/84
- https://github.com/microsoft/DialoGPT/issues/34
- https://www.reddit.com/r/GPT3/comments/uokws7/dialogpt_finetuned_on_my_own_message_data/
- https://chatbotslife.com/creating-cartmanbot-part-two-de95c348aeb1

In [43]:
import argparse
import re

import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    AutoConfig,
    Conversation,
    pipeline,
    ConversationalPipeline,
)


config = AutoConfig.from_pretrained(MODEL_NAME)
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(
    MODEL_NAME,
    from_tf=False,
    config=config,
)

# Let's chat for 4 lines
for step in range(6):
    # encode the new user input, add the eos_token and return a tensor in Pytorch
    new_user_input_ids = tokenizer.encode(input(">> User:") + tokenizer.eos_token, return_tensors='pt')
    # print(new_user_input_ids)

    # append the new user input tokens to the chat history
    bot_input_ids = torch.cat([chat_history_ids, new_user_input_ids], dim=-1) if step > 0 else new_user_input_ids

    # generated a response while limiting the total chat history to 200 tokens,
    chat_history_ids = model.generate(
        bot_input_ids, max_length=200,
        pad_token_id=tokenizer.eos_token_id,
        no_repeat_ngram_size=1,
        do_sample=True,
        top_k=100,
        top_p=0.8,
        temperature=0.8
    )

    # pretty print last ouput tokens from bot
    print("{}".format(tokenizer.decode(chat_history_ids[:, bot_input_ids.shape[-1]:][0], skip_special_tokens=True)))


>> User: I'm happy today


KeyboardInterrupt: 